# Positional Encoding

- Positional Encoding은 embedding된 값에 순서를 부여하기 위해 사용한다.


- RNN 계열 모델과는 다르게 Transformer 모델은 Input sentence를 한번에 모델에 넣기 때문에 모델에게 '순서 정보'를 모델에게 알려 줄 필요가 있다.


- Positional Encoding은 sine함수와 cosine함수가 주기함수인 점에 착안하여 다음과 같은 함수 제시했다.

![image](https://user-images.githubusercontent.com/66320010/150960970-56bd3827-885e-44e0-99dd-d1d1f99eb8bb.png)


- pos는 단어의 위치(position)을 의미하며 i는 해당 pos에 있는 단어의 embedding dimension의 index를 의미한다.

![image](https://user-images.githubusercontent.com/66320010/150973080-e1aacb45-6f13-4be4-8c7c-78db79062901.png)


- embedding dimension의 짝수 index에는 sine함수를, 홀수 index에는 cosine함수를 사용하게 된다.


- 예를 들어, "I have an apple" 이라는 단어가 있고, "I"라는 단어가 [0.1, 0.2, 0.3, 0.4] 로 embedding 되었다고 가정하고, 단어 "I"에 대한 Positional Encoding을 구해보자.


- 단어 "I"는 첫번째에 위치하고 있으므로 pos = 1이 되며, embedding은 총 4개의 dimension으로 되었으니, d_model = 4가 됩니다. 


- 따라서 각 embedding값의 positional encoding은 다음과 같이 구할 수 있다.

![image](https://user-images.githubusercontent.com/66320010/150961750-09b70478-df69-4360-bee7-b2b015564e0e.png)


- 이 결과를 기존 embedding값과 더해주면 모델에 입력으로 들어가는 데이터를 완성할 수 있게 된다.

![image](https://user-images.githubusercontent.com/66320010/150973658-7deb4686-b870-4778-afad-deb3244662a7.png)

In [138]:
import torch
from torch import nn

class PositionalEncoding(nn.Module):
    def __init__(self,max_len,d_model,device):
        super(PositionalEncoding,self).__init__()
        
        self.encoding = torch.zeros(max_len,d_model,device = device)  # max_len * d_model 텐서 생성
        self.encoding.requires_grad = False   # requires_grad = True하면 그 tensor에서 이뤄진 모든 연산들을 추적. False이면 역전파 단계 중에서 이 텐서들에 대한 변화도를 계산할 필요가 없음을 나타냄
        
        pos = torch.arange(0,max_len,device = device)   # torch.arange() : 주어진 범위 내의 정수를 순서대로 생성 / max_len : 각 문장들 중에서 길이가 가장 긴 문장의 단어의 개수
        pos = pos.float().unsqueeze(dim=1)   # unsqueeze() : 특정 위치에 1인 차원을 추가
        
        _2i = torch.arange(0,d_model,step =2, device = device).float()   # step은 간격의미
        
        # [:,1] - 첫번째 차원을 전체 선택한 상황에서 두번째 차원의 첫번째 것만 가져옴
        self.encoding[:,0::2] = torch.sin(pos/(10000**(_2i/d_model)))   # [시작인덱스::증가폭]. 인덱스 0부터 2씩 증가시키면서 마지막 요소까지
        self.encoding[:,1::2] = torch.cos(pos/(10000**(_2i/d_model)))   # 짝수 index엔 sin함수, 홀수 index엔 cos함수 적용
        
    def forward(self,x):
        batch_size, seq_len = x.size()
        
        return self.encoding[:seq_len,:]

# Multi-Head Attention

- 논문에서는 Scale-dot product attention과 같이 설명한다.


- Scale-dot product attention은 입력으로 쿼리,키,밸류를 받은 뒤 해당 문서에서 중요한 단어를 선별하는 작업을 진행한다.


- 이 작업을 진행하기 위해서는 쿼리,키,밸류를 얻는 작업을 진행해야하고 입력 받은 문장의 embedding 크기를 같거나 줄이는 작업을 진행해준다.


- 예시로 "I am a student"를 들면 "I"라는 입력 쿼리에 대해 키로 주어진 각각의 단어들과 내적을 구하여 유사도를 구한다.


- 그 후 scaling을 하고 softmax를 거쳐 밸류 값과 곱해주고 이를 다 더하면 입력 쿼리 "I"에 대한 attention value이다.

![image](https://user-images.githubusercontent.com/66320010/150976745-d2391f0e-fe50-4a14-9fd3-1d3c1fe84b70.png)


- 입력 쿼리는 하나의 문장의 모든 단어에 대해 같은 작업을 수행하기 때문에 단어마다 계산을 하는 것은 비효율적, 그래서 아래 그림처럼 입력 문장의 모든 단어 쿼리에 대해 attention score 구할 수 있다.


![image](https://user-images.githubusercontent.com/66320010/150980039-388f887e-e3fd-4494-81c8-f8b78cdf47a1.png)
  

In [139]:
import math

class ScaleDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaleDotProductAttention,self).__init__()
        self.softmax = nn.Softmax()
        
    def forward(self, q, k, v, mask = None, e = 1e-12):
        batch_size, head, length, d_tensor = k.size()
        
        # 1. dot product Query with Key^T to compute similarity
        k_t = k.view(batch_size, head, d_tensor, length)
        score = (q @ k_t) / math.sqrt(d_tensor)   # "@" 는 코드에서 pytorch의 "matmul" 함수와 기능이 같음
        # q는 [batch_size,head,length,d] , k_t는 [batch_size,head,d,length]의 4차원 텐서
        
        # 2. apply masking
        # decoder에는 2개의 attentioin이 있는데 첫 번째 attention 이름이 masked self attention이고 mask를 선택적으로 사용하는 것
        # 즉, mask를 껐다 켰다 할 수 있도록 조건 만들어 놓은거(필요하면 mask를 씌울 수 있음)
        if mask is not None:
            score = score.masked_fill(mask == 0,-e)      # mask 값이 0이면 -e(작은 값)으로 채워줌, softmax취한 값이 거의 0이 되도록 하기 위함
            
        # 3. pass them softmax     
        score = self.softmax(score)
        
        # 4. multiply with Value
        v = score @ v
        
        return v,score

**패딩 마스크(padding mask)**

- 패딩 마스크는 입력 문장에 < PAD > 토큰이 있을 경우 어텐션에서 사실상 제외하기 위한 방법이다.


- 예를 들어 < PAD >가 포함된 입력 문장의 self attention 예제를 보자. attention을 수행하고 attention score 행렬을 얻는 과정은 다음과 같다.

![image](https://user-images.githubusercontent.com/66320010/151502350-f751fea5-1942-4a94-8e73-08f8d06d5436.png)



- 사실 단어 < PAD >의 경우 실직적인 의미를 가진 단어가 아니다. 그래서 transformer에서는 key의 경우에 < PAD > 토큰이 존재한다면 이에 대해서는 유사도를 구하지 않도록 masking을 해주기로 했다(여기서 masking이란 attention에서 제외하기 위해 값을 가린다는 의미이다).


- attention score 행렬에서 행에 해당하는 문장은 query이고, 열에 해당하는 문장은 key이다. 그리고 key에 < PAD >가 있는 경우는 해당 열 전체를 masking 해준다.


![image](https://user-images.githubusercontent.com/66320010/151502612-cd9d5483-1b64-4154-82a8-fcbf8528b826.png)



- masking하는 방법은 attention score 행렬의 마스킹 위치에 매우 작은 음수값을 넣어주는 것이다. 여기서 매우 작은 음수값이라는 것은 -1,000,000,000과 같은 -무한대에 가까운 수라는 의미이다. 



- 현재 attention score함수는 softmax함수를 지나지 않은 상태이다. 원래 attention score함수는 softmax함수를 지나고 그 후 value행렬과 곱해지게 된다.


- 그런데 masking위치에 매우 작은 음수 값들이 들어가 있으면 attention score행렬이 소프트맥스 함수를 지난 후에는 해당 위치 값은 0이 되어 단어간 유사도를 구하는 일에 < PAD > 토큰이 반영되지 않게 된다.

![image](https://user-images.githubusercontent.com/66320010/151502888-58526e89-c211-4daa-9c4e-c2009b52caf3.png)


- 패딩 마스크를 구현하는 방법은 입력된 정수 시퀀스에서 패딩 토큰의 인덱스인지, 아닌지 함수를 구현하는 것이다. 정수 시퀀스에서 0인 경우는 1로 변환하고 그렇지 않은 경우에는 0으로 변환하는 함수이다.

**멀티 헤드 어텐션(multi-head attention)**

- Transformer 연구진들은 Scale dot product를 단일로 이용하지 않고 병렬로 이용하는 multi-head attention을 사용하였다.


- 논문에 따르면, 단일 attention을 사용했을 때보다 multi-head attention을 사용하였을 때 문장의 특징 정보를 더 많이 잡을 수 있었다고 한다.


- 병렬 attention 연산을 수행한 뒤, multi-head attention에서는 각 연산 결과를 합치는(concaternate)과정을 진행한다.

![image](https://user-images.githubusercontent.com/66320010/150980783-9b957d6b-dbb9-4930-9976-4c9d56620b53.png)


- 합친 행렬의 크기는 (seq_len, d_model)이 된다. 여기에 가중치행렬(W)를 곱하면 multi-head attention의 최종 출력이 된다. 


- 이 때 가중치 행렬 W의 shape은 (d_model,d_model)이다.


- 쉽게 설명하면, d_model의 차원을 num_heads개로 나누어서 d_model / num_heads의 차원을 갖는 쿼리,키,밸류에 대해 num_heads개의 병렬 어텐션을 수행하는 것이다.

![image](https://user-images.githubusercontent.com/66320010/150982398-fb617105-6408-4b7a-bf44-02aaef360432.png)


- 입력 문장이 들어오면 h개의 서로 다른 쿼리,키,밸류로 구분될 수 있도록 한다. 그 이유는 h개의 서로 다른 어텐션 컨셉을 학습하도록해서 더욱된 구분된 특징을 학습할 수 있게 하기 위해서이다.

![image](https://user-images.githubusercontent.com/66320010/150988415-3f35a1a6-fee9-4406-989f-a4e100d48394.png)


- 입력으로 들어온 V,K,Q는 linear 계층으로 보낸다. 이 linear 계층은 완전연결층을 의미한다. 


- 위 이미지를 보면 입력 데이터로 2x4 shape의 데이터가 들어가지만 여러개의 가중치(위 이미지에서는 0-7까지 총 8개)로 연산을 하면서 데이터가 8개로 쪼개진다(Q0-7,K0-7, V0-7 이런식으로).


- 또한 2x4 shape과 가중치 shape인 4x3이 행렬곱을 하므로 최종적으로 출력되는 데이터의 shape는 2x3이다.


- 이것을 scaled-dot production attention에 입력하게되면 연산 과정에 의해 같은 shape인 2x3이 나오게 되고 이것이 그림에서 Z0~7을 의미한다.


- 이것들을 concat을 하게 되면 2x3 shape인 데이터들이 가로로 붙으면서 2x24 shape이 되고 이를 가중치 W(24x4)와 연산하면 최종적인 출력인 Z(2x4)가 나온다.

In [140]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_model,n_head):
        super(MultiHeadAttention,self).__init__()
        
        self.n_head = n_head
        self.attention = ScaleDotProductAttention()
        self.w_q = nn.Linear(d_model,d_model)   # 위 그림에서 쿼리,키,밸류가 처음에 각각 linear층을 거치게 되는데 그 부분 의미
        self.w_k = nn.Linear(d_model,d_model)
        self.w_v = nn.Linear(d_model,d_model)
        self.w_concat = nn.Linear(d_model,d_model)
     

    # split 함수를 통해 W^Q, W^k, W^V를 head 개수별로 쪼개줌    
    def split(self,tensor):
        batch_size, length, d_model = tensor.size()
        d_tensor = d_model / self.n_head
        tensor = tensor.view(batch_size, self.n_head, length, d_tensor)
        
        return tensor
    
    def concat(self,tensor):
        batch_size, head, length, d_tensor = tensor_size()
        d_model = head * d_tensor
        
        tensor = tensor.view(batch_size, length, d_model)
        
        return tensor
    
    def forward(self, q, k ,v, mask=None):     # q,k,v shape : [batch_size, query/key/value_len, d_model]
        # 1. dot product with weight matrices
        q,k,v = self.w_q(q), self.w_k(k), self.w_v(v)
        
        # 2. split tensor by number of heads
        q,k,v = self.split(q), self.split(k), self.split(v)
        
        # 3. do scale dot product to compute similarity
        out, attention = self.attention(q,k,v,mask = mask)
        
        # 4. concat and pass to linear layer
        out = self.concat(out)
        out = self.w_concat(out)
        
        return out

**정리하면 multi head attention은 크게 다섯 가지 파트로 구성된다.**

1) W^Q, W^k, W^V에 해당하는 d_model 크기의 dense layer를 지나게 한다.

2) 지정된 헤드 수(num_heads)만큼 나눈다(split).

3) scaled dot product attention 수행한다.

4) 나눠졌던 헤드들을 연결(concatenation)한다.

5) W^O에 해당하는 dense layer를 지난다.

# Add & Norm

- residual connection을 한 이후에 layer normalization을 수행한다.


- ResNet과 같이 이전 데이터를 단순 합산(Add)하며 기존의 데이터에 대한 정보를 잃지 않고 Normalization(정규화)를 하는 계층이다.

![image](https://user-images.githubusercontent.com/66320010/150990483-1dbdca20-bea7-468a-9a20-fbec85175955.png)


- batch normalization은 batch 통계량을 사용하므로 하나의 batch가모두 계산되기 전까지는 다음 단계로 넘어갈 수 없고, RNN에 적용하기 까다롭다는 단점이 있다.


- layer normalization은 batch normalization의 단점을 보완하면서 학습 속도를 빠르게 하는데 장점이 있는 기법이다.

![image](https://user-images.githubusercontent.com/66320010/151122032-bb218f84-2583-43a3-92d7-96af24b34639.png)


- layer normalization은 미니배치 축이 아닌 채널 축에 대해 다음 식을 이용하여 정규화 수행한다(epsilon은 부모가 0이 되는것을 방지해주는 상수정도로 이해하면 됨).

![image](https://user-images.githubusercontent.com/66320010/151122566-d5d84d56-91a4-47e5-9d56-fe82473da6c6.png)


- 위 결과에 gamma와 beta를도입하여 다음과 같은식을 세워주며 gamma와 beta값은 1과 0으로 설정해주면 정규화가 마무리된다.

![image](https://user-images.githubusercontent.com/66320010/151122996-82a7631b-2dc3-4dec-8ab4-9483239d2eac.png)

In [141]:
class LayerNorm(nn.Module):
    def __init__(self,d_model,eps = 1e-12):
        super(LayerNorm,self).__init__()
        
        self.gamma = nn.Parameter(torch.ones(d_model))   # nn.Parameter - tensor의 한 종류로, Module에 속성으로 할당될 때 자동으로 매개변수로 등록됨
        self.beta = nn.Parameter(torch.zeros(d_model))
        self.eps = eps
        
    def forward(self,x):
        mean = x.mean(-1,keepdim=True)    # "-1" means last dimension
        std = x.std(-1,keepdim=True)
        
        out = (x-mean) / (std+self.eps)
        out = self.gamma * out + self.beta
        
        return out

# (Position-wise) Feed-Foward

- Feed-Forward는 완전 연결 신경망으로 Transformer의 encoder와 decoder 모두에 사용하는 sub-layer이다.


- 단순하게 2개의 FC layer를 갖는 레이어이다. multi-head attention lyaer의 output을 받아 연산을 수행하고 그 결과를 다음 encoder에게 넘겨준다.


- 논문에서 사용한 FFN의 수식은 다음과 같다.

![image](https://user-images.githubusercontent.com/66320010/151125720-83fb4a94-921a-4d93-bade-bdde2190c0a6.png)

In [142]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self,d_model,hidden, drop_prob=0.1):
        super(PositionwiseFeedForward,self).__init__()
        
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p = drop_prob)
        
    def forward(self,x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        
        return x

# Encoder

- Encoder는 multi-head attention, add & norm, feed-forward, add & norm 순으로 네트워크가 이어진다.


- 하나의 encoder layer를 사용하지 않고 그림에서는 N개의 encoder layer를 사용한다. 따라서 구현 시에는 하나의 encoder layer를 구현한 뒤 N개의 encoder가 쌓여있는 encoder를 구현하는 방식으로 구현할 수 있다.

In [143]:
class EncoderLayer(nn.Module):
    def __init__(self,d_model, ffn_hidden, n_head, drop_prob):
        super(EncoderLayer,self).__init__()
        
        self.attention = MultiHeadAttention(d_model, n_head)
        self.norm1 = LayerNorm(d_model = d_model)
        self.dropout1 = nn.Dropout(p = drop_prob)
        self.ffn = PositionwiseFeedForward(d_model = d_model, hidden = ffn_hidden, drop_prob = drop_prob)
        
        self.norm2 = LayerNorm(d_model = d_model)
        self.dropout2 = nn.Dropout(p = drop_prob)
        
    def forward(self,x,src_mask):    # src_mask : encoder의 입력 sequence중 attention 계산을 수행하지 않고 무시할 sequence 정보를 제공하는 mask값을 갖는 sequence
        _x = x
        
        # 1. compute multi-head attention
        x = self.attention(q=x, k=x, v=x, mask = src_mask)  # 입력값(x)가 들어오면 그 값을 키,쿼리,밸류에 복사해서 그대로 넣어줌, src_mask는 특정 단어에 대해 attention을 수행하지 않도록 하기 위한 parameter
        
        # 2. compute add & norm
        x = self.norm1(x + _x)   # residual connection한 뒤 normalization
        x = self.dropout(x)
        
        # 3. compute feed-forward network
        _x = x
        x = self.ffn(x)
        
        # 4. compute add & norm
        x = self.norm2(x + _x)
        x = self.dropout2(x)   
        
        return x

- 이렇게 하나의 encoder layer를 구현했으면, 다음으로 N개의 encoder layer를 이어붙일 차례이다.


- encoder에 입력으로 들어가는 문장은 embedding을 거치고 positional encoding과 합쳐서 들어가기 때문에 코드에 이 부분도 추가한다.

In [319]:
class Encoder(nn.Module):
    def __init__(self, enc_voc_size, max_len, ffn_hidden, n_head, n_layers, drop_prob, device):
        super().__init__()
        
        # embedding
        self.embed = nn.Embedding(num_embeddings = len(src_.vocab), embedding_dim= d_model, padding_idx = 1)   # 실제 단어들의 갯수에 해당하는 num_embeddings이 들어오면 d_model(임베딩 차원)으로 바꿔줌
        
        # positional encoding
        self.pe = PositionalEncoding(max_len = max_len, d_model = d_model, device = device)
        
        # add multi layers
        self.layers = nn.ModuleList([EncoderLayer(d_model=d_model, ffn_hidden = ffn_hidden, n_head = n_head, drop_prob = drop_prob) for _ in range(n_layers)])
        
    def forward(self,x,src_mask):
        # compute embedding
        x = self.embed(x)
        
        # get positional encoding
        x_pe = self.pe(x)
        
        # embedding + positional encoding
        x = x + x_pe
        
        # compute encoder layers
        for layer in self.layers:
            x = layer(x, src_mask)
            
        # return encoder output
        return x

# Decoder

- encoder와는 다르게 decoder에서는 총 2번의 attention이 수행된다.


- decoder의 첫 번째 attention 계산에서는 decoder 입력만으로 attention이 계산되지만 두 번째 attention에서는 첫 번째 attention 계산과는 다르게 첫 번째 attention 결과 값이 query로 들어가며 key와 value는 마지막 encoder의 출력값이 들어간다.


- add & norm 부분과 feed-forward는 encoder에서 수행한 것과 동일한 메커니즘으로 수행된다.

![image](https://user-images.githubusercontent.com/66320010/151138608-0f0ed069-097a-46ce-a888-a14bde5b6f35.png)

In [320]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, n_head, drop_prob):
        super(DecoderLayer,self).__init__()
        
        # self attention(only Decoder input)
        self.self_attention = MultiHeadAttention(d_model = d_model, n_head = n_head)
        
        # layer normalization
        self.norm1 = LayerNorm(d_model = d_model)
        self.dropout1 = nn.Dropout(p=drop_prob)
        
        # attention (encoder + decoder)
        self.enc_dec_attention = MultiHeadAttention(d_model = d_model, n_head = n_head)
        
        # layer normalization
        self.norm2 = LayerNorm(d_model = d_model)
        self.dropout2 = nn.Dropout(p=drop_prob)
        
        # feed-forward
        self.ffn = PositionwiseFeedForward(d_model = d_model, hidden = ffn_hidden, drop_prob = drop_prob)
        
        # layer normalization
        self.norm3 = LayerNorm(d_model = d_model)
        self.dropout3 = nn.Dropout(p = drop_prob)
        
    def forward(self,dec,enc, trg_mask, src_mask):    # trg_mask : decoder의 입력 sequence 중 attention 계산을 수행하지 않고 무시할 sequence 정보를 제공하는 mask값을 갖는 sequence
        _x = dec
        
        # compute self-attention
        x = self.self_attention(q=dec, k=dec, v=dec, mask = trg_mask)   # 2개의 attention중 첫 번째 attention은 쿼리,키,밸류에 자기자신(dec)을 넣음
        
        # compute add & norm
        x = self.norm1(x + _x)
        x = self.dropout1(x)
        
        if enc is not None:    # encoder의 출력값이 있다면(없으면 FFN으로 넘어감)
            _x = x
            
            # Compute encoder - decoder attention
            # Query(q) : decoder attention output
            # Key(k) : Encoder output
            # Value(v) : Encoder output
            x = self.enc_dec_attention(q = x, k = enc, v = enc, mask = src_mask)  # 쿼리는 디코더에 포함된 출력된 단어들에 대한 정보이고 키와 밸류는 enc에서 마지막에 나온 값 사용
            
            # compute add & norm
            x = self.norm2(x + _x)
            x = self.dropout2(x)
            
        _x =x
        
        # compute FFN
        x = self.ffn(x)
        
        # compute add & norm
        x = self.norm3(x+_x)
        x = self.dropout3(x)
        
        return x

- encoder를 구현한 것과 같은 방식으로 위에서 구현한 decoder layer를 여러개로 이어서 하나의 decoder 객체로 만들어준다.


- encoder를 구현할 때와 마찬가지로, embedding 부분과 positional encoding 부분도 추가해준다.

In [321]:
class Decoder(nn.Module):
    def __init__(self,dec_voc_size, max_len, d_model, ffn_hidden, n_head, n_layers, drop_prob, device):
        super().__init__()
        
        # embedding
        self.embed = nn.Embedding(num_embeddings = len(trg_.vocab), embedding_dim = d_model, padding_idx = 1)   # 단어의 개수와 같은 dimension -> embedding 차원으로
        
        # positional encoding
        self.pe = PositionalEncoding(max_len = 50, d_model = d_model, device = device)
        
        # add decoder layer
        self.layers = nn.ModuleList([DecoderLayer(d_model = d_model, ffn_hidden = ffn_hidden, n_head = n_head, drop_prob = drop_prob) for _ in range(n_layers)])
        
        # linear
        self.linear = nn.Linear(d_model, dec_voc_size)
        
    def forward(self,trg,src,trg_mask,src_mask):
        
        # compute embedding
        trg = self.embed(trg)
        
        # get positional encoding
        trg_pe = self.pe(trg)
        
        # embedding + positional encoding
        trg = trg + trg_pe
        
        # compute decoder layers
        for layer in self.layers:
            trg = layer(trg,srg,trg_mask, src_mask)   # srg : enc에서 나온 마지막 output값..?
            
        output = self.linear(trg)
        
        return output

# Transformer

- 위에서 구현한 Encoder와 Decoder를 하나의 모델로 합쳐준다.

In [322]:
class Transformer(nn.Module):
    def __init__(self, src_pad_idx, trg_pad_idx, enc_voc_size, dec_voc_size, d_model, n_head, max_len, ffn_hidden, n_layers, drop_prob,device):
        super().__init__()
        
        self.src_pad_idx = src_pad_idx    # encoder 입력에서 mini-batch단위로 들어오게 되는데 mini-batch 내 길이가 짧은 sequence에 padding을 수행하게 되는데 이 padding에는 attention을 수행하지 않도록 padding의 index에는 mask를 수행하는 mask sequence
        self.trg_pad_idx = trg_pad_idx    # decoder 입력에서 padding에는 attention을 수행하지 않도록 padding index에는 mask를 수행하는 mask sequence
#         sefl.trg_sos_idx = trg_sos_idx   어디에 쓰이는지 모르겠음. 파라미터에서도 지움
        
        # Encoder
        self.encoder = Encoder(enc_voc_size = enc_voc_size, max_len = max_len, ffn_hidden = ffn_hidden, n_head = n_head, n_layers = n_layers, drop_prob = drop_prob, device = device)
        
        # Decoder
        self.decoder = Decoder(dec_voc_size = dec_voc_size,max_len = max_len, d_model = d_model, ffn_hidden = ffn_hidden, n_head = n_head, n_layers = n_layers, drop_prob = drop_prob, device = device)
        
        self.device = device
        
    # mask는 padding index를 False로 만들고, 글자 부분은 True로 만듦
    # 이후 no peaking mask와 동일한 형태를 가지기 위해서 차원을 하나 더 더해줌
    def make_pad_mask(self,q,k):   # padding 부분은 attention연산에서 제외해야하므로 mask를 씌워줘서 계산이 되지 않도록 함
        len_q, len_k = q.size(1), k.size(1)
        print(len_k)
        
        # batch_size x 1 x 1 x len_k
        k = k.ne(self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        print(k.shape)
        
        # batch_size x 1 x len_1 x len_k
        k = k.repeat(1,1,len_q,1)
        
        # batch_size x 1 x len_q x 1
        q = q.ne(self.src_pad_idx).unsqueeze(1).unsqueeze(3)
        
        # batch_size x 1 x len_q x len_k
        q = q.repeat(1,1,1,len_k)
        
        mask = k & q
        
        return mask
    
    # decoder 부분에서 t번째 단어를 예측하기 위해 입력으로 t-1단어까지 넣어야 하므로 나머지 부분을 masking처리함.
    # 만약 t번째 단어를 예측하는데 이미 decoder에 t번째 단어가 들어간다면 ? ==> 답을 이미 알고 있는 상황
    # 따라서 seq2seq 모델에서 처럼 t번째 단어를 예측하기 위해서 t-1번째 단어까지만 입력될 필요가 있음
    # t+1부터 ~~ max_len 까지 단어는 t번째 단어를 예측하는데 전혀 필요하지 않음 ==> masking!!
    def make_no_peak_mask(self,q,k):   
        len_q, len_k = q.size(1), k.size(1)
        
        # len_q x len_k
        # 마스크 예시 - 아래 삼각형은 다 1이고 위 삼각형은 다 0인 마스크(앞 쪽 단어만 볼 수 있도록 만든 마스크)
        """ (마스크 예시)
        1 0 0 0 0
        1 1 0 0 0
        1 1 1 0 0
        1 1 1 1 0
        1 1 1 1 1
        """
        mask = torch.tril(torch.ones(len_q, len_k)).type(torch.BoolTensor).to(self.device)   # torch.tril - 하삼각행렬
        
        return mask
    
    def forward(self, src, trg):
        # get mask
        src_mask = self.make_pad_mask(src,src)
        src_trg_mask = self.make_pad_mask(trg,src)    # decoder의 enc-dec attention에 입력되는 sequence중 attention 계산을 수행하지 않고 무시할 sequence 정보를 제공하는 mask값을 갖는 sequence..?
        trg_mask = self.make_pad_mask(trg,trg) * self.make_no_peak_mask(trg,trg)
        
        # compute encoder
        enc_src = self.encoder(src,src_mask)   # 인코더의 출력값 뽑아서
        
        # compute decoder
        output = self.decoder(trg,enc_src, trg_mask, src_trg_mask)   # 이걸 가지고 디코더에서 attention할 수 있게 한 것
        
        return output

**mask_no_peak_mask 함수 관련 보충설명**

- 디코더의 self attention은 "masked" multi head attention이다.


- 왜 인코더와 다르게 "masked"가 붙었을까? 일반적으로 디코더는 학습할 때에 이전의 출력값에 의존한다. 그러나 이전의 값에 오류가 있다면 그 오류에 의존하여 계속 잘못된 디코딩이 진행된다.


- 이러한 것을 막기 위해 이전 값은 정답으로부터 가져오도록 하는 것이 teacher forcing 기법이다.

![image](https://user-images.githubusercontent.com/66320010/151493164-a445ffc8-d3ff-49b4-981e-2e2ed0d044d0.png)


- 이러한 teacher forcing방법이 transformer에 그대로 적용될 수 있을까? 정답은 '아니다'이다.


- teacher forcing의 예시로 든 그림은 RNN기반 모델이며 RNN기반 모델은이전 cell의 output을 이후 cell에서 사용할 수 있다.


- 하지만 transformer는 벙렬 연산을 수행하므로 ground truth의 embedding을 matrix로 만들어 input으로 그대로 사용하게 되면, 디코더에서 self attention 연산을 수행하게 될 때 현재 출력해내야하는 token의 정답까지 알고 있는 상황이 된다.



- 따라서 masking을 적용해야한다. i번째 token을 생성해낼 때, 1 ~ i-1의 token만 보이게 처리를 해야한다. 이러한 마스킹 기법을 subsequent masking이라고 한다.

# 참고

https://kaya-dev.tistory.com/8

https://kaya-dev.tistory.com/11

https://hongl.tistory.com/194

https://wikidocs.net/31379

# Example 

- 독일어(Deutsch)을 영어(English)로 번역하는 예시


- 번역된 영어 문장의 성능을 평가하기 위한 척도로 BLEU score 사용


- BLEU score는 번역된 문장이 실제 정답 문장과 비교했을 때 얼마나 유사한지 평가해주는 평가척도 ==> 이를 사용하기 위해 torchtext를  특정버전(0.6.0)으로 설치

## Data preprocessing(데이터 전처리)

- spaCy 라이브러리 : 문장의 토큰화(tokenization), 태깅(tagging)등의 전처리 기능을 위한 라이브러리


- spaCy를 설치한 후에는 언어에 맞는 모델도 설치를 해야 한다(tokenizing, parsing, pos tagging 등을 하기 위한 모델).


- 독일어와 영어 전처리 모듈 설치

In [323]:
import spacy

# 문장을 토큰으로 바꿔줄 수 있는 spacy 라이브러리 객체 생성
spacy_en = spacy.load('en_core_web_sm')   # 영어 토큰화(tokenization)   
spacy_de = spacy.load('de_core_news_sm')   # 독일어 토큰화(tokenization)

tokenized = spacy_en.tokenizer("I am a graduate student.")   # 토큰화 기능 써보기

for i, token in enumerate(tokenized):
    print(f"인덱스 {i} : {token.text}")  # token.text은 str형태로 바꿔준거
    
print(tokenized)

인덱스 0 : I
인덱스 1 : am
인덱스 2 : a
인덱스 3 : graduate
인덱스 4 : student
인덱스 5 : .
I am a graduate student.


- 영어 및 독일어 **토큰화 함수** 정의

In [324]:
# 네트워크에 입력값으로 넣을 수 있도록 토큰화


# 독일어 문장을 토큰화 하는 함수
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]   # print(spacy_de.tokenizer) - 문장이"I love you"이면 I love you 출력


# 영어 문장을 토큰화 하는 함수
def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]   # return - 문장이 "I love you"이면 ['I','love','you'] 반환

- 필드(field) 라이브러리를 이용해 데이터셋에 대한 구체적인 전처리 내용을 명시 ==> 데이터 셋이 있을 때 그 데이터 셋에 대해서 어떻게 전처리 할건지 명시할 수 있도록함


- seq2seq 모델과는 다르게 batch_first 속성 값을 True로 설정


- 번역 목표
      
     소스(src):독일어
     
     목표(trg):영어

In [325]:
from torchtext.data import Field,BucketIterator

# 문장의 맨 앞에는 <sos>토큰, 맨 뒤에는 <eos>토큰을 붙임, 각각의 단어는 소문자로 변환이 일반적, 트랜스포머에 넣을 때에는 텐서의 차원에서 시퀀스보단 배치가 먼저 오도록함
src_ = Field(tokenize = tokenize_de, init_token = "<sos>", eos_token = "<eos>", lower = True, batch_first=True)
trg_ = Field(tokenize = tokenize_en, init_token = "<sos>", eos_token = "<eos>", lower = True, batch_first=True)

- 대표적인 영어-독일어 번역 데이터셋을 불러온다(Multi30k)

In [326]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".de",".en"),fields=(src_,trg_))   # exts: 각 언어에 대한 경로 확장을 포함하는 튜플 / fields :라이브러리를 이용해 독일어->영어 task에 대해서 앞서 정의한 전처리를 수행할 수 있도록함

In [327]:
# data 개수 출력
print(f"train dataset 크기: {len(train_dataset.examples)}개")
print(f"validation dataset 크기: {len(valid_dataset.examples)}개")
print(f"test dataset 크기: {len(test_dataset.examples)}개")

# train 데이터 중 하나 선택해 출력
print(vars(train_dataset.examples[10]))   # 출력하고 싶은 값이 object인 경우 vars라는 내장함수를 사용하여 내용 볼 수 있음

train dataset 크기: 29000개
validation dataset 크기: 1014개
test dataset 크기: 1000개
{'src': ['eine', 'ballettklasse', 'mit', 'fünf', 'mädchen', ',', 'die', 'nacheinander', 'springen', '.'], 'trg': ['a', 'ballet', 'class', 'of', 'five', 'girls', 'jumping', 'in', 'sequence', '.']}


- 필드(field) 객체의 build_vocab 메서드를 이용해 영어와 독일어의 단어 사전을 만든다.


- 이렇게 해주는 이유는 독일어->영어 번역 할 때, 각각의 초기 인풋 차원이 얼마인지를 구할 수 있기 때문이다. 


- 전체 단어들 중 최소 2번 이상 등장한 단어만 선택한다.

In [328]:
src_.build_vocab(train_dataset, min_freq = 2)   # 단어 집합 생성, min_freq는 단어 집합(사전)에 추가시 단어의 최소 등장 빈도 조건을 추가, max_size는 단어 집합의 최대 크기를 지정
trg_.build_vocab(train_dataset, min_freq = 2)   
# 왜 바로 src, trg.build_vocab 이렇게 쓸 수 있는건지..?

# print(trg.vocab.stoi)   # 생성된 단어 집합(사전) 내의 단어는 .stoi를 통해서 확인 가능

print(f"len(src):{len(src_.vocab)}")
print(f"len(trg):{len(trg_.vocab)}")

print(trg_.vocab.stoi["hello"])  # 각각의 단어가 어떤 인덱스에 해당하는지 볼 수 있음
print(trg_.vocab.stoi["hhhh"])  # 존재하지 않는 단어 : 0
print(trg_.vocab.stoi[trg_.pad_token])   # padding : 1
print(trg_.vocab.stoi["<sos>"])  # <sos> : 2
print(trg_.vocab.stoi["<eos>"])  # <eos> : 3

len(src):7853
len(trg):5893
4112
0
1
2
3


- 한 문장에 포함된 단어가 순서대로 나열된 상태로 네트워크에 입력되어야한다.


- 따라서 하나의 배치에 포함된 문장들이가지는단어의 개수가 유사하도록 만들면 좋다.


- 이를 위해 BucketIterator를 사용한다.


- batch_size = 128

In [329]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 128

# BucketIterator는 모든 텍스트 작업을 일괄로 처리하고 단어를 인덱스 숫자로 변환하는 것을 도움
train_iterator, valid_iterator, test_iterator = BucketIterator.splits((train_dataset, valid_dataset, test_dataset), batch_size = batch_size, device = device)

In [330]:
for i,batch in enumerate(train_iterator):
    src = batch.src
    trg = batch.trg
    
    print("첫 번째 배치 크기: {}".format(src.shape))
    
    for i in range(src.shape[1]):  # 현재 배치에 있는 하나의 문장에 포함된 정보 출력, src.shape[1] : sequence length
        print(f"인덱스 {i}: {src[0][i].item()}")   # src[0][i] : 첫 번째 문장에 있는 각각의 단어를 출력 ,   [seq_num,seq_len]
        
    break   # 첫 번째 배치만 확인

첫 번째 배치 크기: torch.Size([128, 25])
인덱스 0: 2
인덱스 1: 8
인덱스 2: 16
인덱스 3: 10
인덱스 4: 5
인덱스 5: 49
인덱스 6: 9
인덱스 7: 470
인덱스 8: 271
인덱스 9: 358
인덱스 10: 9
인덱스 11: 74
인덱스 12: 17
인덱스 13: 34
인덱스 14: 72
인덱스 15: 4
인덱스 16: 3
인덱스 17: 1
인덱스 18: 1
인덱스 19: 1
인덱스 20: 1
인덱스 21: 1
인덱스 22: 1
인덱스 23: 1
인덱스 24: 1


## Train

- 학습 파라미터 설정 및 모델 초기화

In [331]:
input_dim = len(src_.vocab)   # src언어에 포함된 단어의 개수
output_dim = len(trg_.vocab)
d_model = 512   # embedding차원
n_layer = 3
n_head = 8
max_len = 40
ffn_hidden = 128
drop_prob = 0.1
epochs = 50

In [332]:
src_pad_idx_ = src_.vocab.stoi[src_.pad_token]
trg_pad_idx_ = trg_.vocab.stoi[trg_.pad_token]

# 인코더와 디코더 객체 선언
enc = Encoder(input_dim, max_len, ffn_hidden, n_head, n_layer, drop_prob, device)
dec = Decoder(output_dim, max_len, d_model, ffn_hidden,n_head, n_layer, drop_prob, device)

# 트랜스포머 객체 선언
model = Transformer(src_pad_idx_, trg_pad_idx_, input_dim, output_dim, d_model, n_head, max_len, ffn_hidden, n_layer, drop_prob, device).to(device)

- 모델 가중치 파라미터 초기화

In [333]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,322,309 trainable parameters


In [334]:
# 전체 네트워크에 포함되어있는 파라미터 확인 가능
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (embed): Embedding(7853, 512, padding_idx=1)
    (pe): PositionalEncoding()
    (layers): ModuleList(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (attention): ScaleDotProductAttention(
            (softmax): Softmax(dim=None)
          )
          (w_q): Linear(in_features=512, out_features=512, bias=True)
          (w_k): Linear(in_features=512, out_features=512, bias=True)
          (w_v): Linear(in_features=512, out_features=512, bias=True)
          (w_concat): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNorm()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=128, bias=True)
          (linear2): Linear(in_features=128, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNorm()
        (dropo

- 학습 및 평가 함수 정의

In [335]:
import torch.optim as optim

# Adam optimizer로 학습 최적화
learning_rate = 0.0005
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

criterion = nn.CrossEntropyLoss(ignore_index = trg_pad_idx_)   # 뒷 부분의 padding에 대해서는 값 무시

In [336]:
# 학습(train) 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train()   # 학습 모드
    epoch_loss = 0
    
    # 전체 학습 데이터를 확인하며
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        # 출력 단어의 마지막 인덱스(<eos>)는 제외
        # 입력을 할 때는 <sos>부터 시작하도록 처리
        output, _ = model(src,trg[:,:-1])
        
        output_dim = output.shape[-1]
        
        output = output.contiguous().view(-1,output_dim)
        
        # 출력 단어의 인덱스 0(<sos>)은 제외
        trg = trg[:,1:]
        
        # 모델의 출력결과와 타겟 문장을 비교하여 손실 계산
        loss = criterion(output, trg)
        loss.backward()   # 기울기(gradient) 계산
        
        # 기울기 clipping진행
        torch.nn.utils.clip_grad_norm(model.parameters().clip)
        
        # 파라미터 업데이트
        optimizer.step()  
        
        # 전체 손실 계산 
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [337]:
# 모델 평가(evaluate) 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad():
        # 전체 평가 데이터를 확인하며
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            # 출력 단어의 마지막 인덱스(<eos>)는 제외
            # 입력을 할 때는 <sos>부터 시작하도록 처리
            output, _ = model(src, trg[:,:-1])

            # output: [배치 크기, trg_len - 1, output_dim]
            # trg: [배치 크기, trg_len]

            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # 출력 단어의 인덱스 0(<sos>)은 제외
            trg = trg[:,1:].contiguous().view(-1)

            # output: [배치 크기 * trg_len - 1, output_dim]
            # trg: [배치 크기 * trg len - 1]

            # 모델의 출력 결과와 타겟 문장을 비교하여 손실 계산
            loss = criterion(output, trg)

            # 전체 손실 값 계산
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

- 학습 및 검증 진행

In [338]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [339]:
import time
import math
import random

CLIP = 1
best_valid_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:   # validation loss가 감소하는 경우에만 모델 저장하게 함
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')
    
    
    
# 오류 - 현재 2개의 값까지만 가능한데 그 이상의 값을 갖고 있다는 의미

31
torch.Size([128, 1, 1, 31])
31
torch.Size([128, 1, 1, 31])
26
torch.Size([128, 1, 1, 26])


ValueError: too many values to unpack (expected 2)

- 모델 test 결과 확인

In [ ]:
model.load_state_dict(torch.load('transformer_german_to_english.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):.3f}')

## BLEU Score 계산

- 학습이 완료된 트랜스포머 모델의 BLEU 스코어 계산

In [ ]:
rom torchtext.data.metrics import bleu_score

def show_bleu(data, src_field, trg_field, model, device, max_len=50):
    trgs = []
    pred_trgs = []
    index = 0

    for datum in data:
        src = vars(datum)['src']
        trg = vars(datum)['trg']

        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len, logging=False)

        # 마지막 <eos> 토큰 제거
        pred_trg = pred_trg[:-1]

        pred_trgs.append(pred_trg)
        trgs.append([trg])

        index += 1
        if (index + 1) % 100 == 0:
            print(f"[{index + 1}/{len(data)}]")
            print(f"예측: {pred_trg}")
            print(f"정답: {trg}")

    bleu = bleu_score(pred_trgs, trgs, max_n=4, weights=[0.25, 0.25, 0.25, 0.25])
    print(f'Total BLEU Score = {bleu*100:.2f}')

    individual_bleu1_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1, 0, 0, 0])
    individual_bleu2_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 1, 0, 0])
    individual_bleu3_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 0, 1, 0])
    individual_bleu4_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[0, 0, 0, 1])

    print(f'Individual BLEU1 score = {individual_bleu1_score*100:.2f}') 
    print(f'Individual BLEU2 score = {individual_bleu2_score*100:.2f}') 
    print(f'Individual BLEU3 score = {individual_bleu3_score*100:.2f}') 
    print(f'Individual BLEU4 score = {individual_bleu4_score*100:.2f}') 

    cumulative_bleu1_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1, 0, 0, 0])
    cumulative_bleu2_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/2, 1/2, 0, 0])
    cumulative_bleu3_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/3, 1/3, 1/3, 0])
    cumulative_bleu4_score = bleu_score(pred_trgs, trgs, max_n=4, weights=[1/4, 1/4, 1/4, 1/4])

    print(f'Cumulative BLEU1 score = {cumulative_bleu1_score*100:.2f}') 
    print(f'Cumulative BLEU2 score = {cumulative_bleu2_score*100:.2f}') 
    print(f'Cumulative BLEU3 score = {cumulative_bleu3_score*100:.2f}') 
    print(f'Cumulative BLEU4 score = {cumulative_bleu4_score*100:.2f}') 

In [ ]:
show_bleu(test_dataset, src_, trg_, model, device)  # 문장 100개 당 한번씩 예측과 정답을 출력하도록함

## 참고

https://youtu.be/AA621UofTUA